<a href="https://colab.research.google.com/github/sathyanarayanajammala/GenAI/blob/main/Additional_NB_01_LangChain_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative AI and Prompt Engineering
## A programme by IISc and TalentSprint
### Additional Notebook: LangChain Agents 🦜🔗

(Ungraded)

## Learning Objectives

At the end of the experiment, you will be able to:

* understand the difference between a tool and an agent
* use Agents in LangChain
* add different tools to your agent such as wikipedia tool, math calculator, google search, python executor, etc
* define your own tool to add to an agent

## Information

> **What are Tools?**
>
> Tools are helpful and act as a interface that an agent, a chain, or a chat model / LLM (Large Language Model) can use to ineract with the world. For e.g., Tools can be Search tool like SerpAPI, Google search etc.,. Given a list of tools and prompt, an LLM can request that one or more tools be invoked with appropriate arguments.

> **Whar are Agents?**
>
> In LangChain, an “**Agent**” is an AI entity that interacts with various “**Tools**” to perform tasks or answer queries. Tools are essentially functions that extend the agent's capabilities by allowing it to perform specific actions, like retrieving the current time or accessing an external database.

<img src='https://drive.google.com/uc?id=1Uo1YNVICwjpKm6PTZCdRGpuSKvTGL6q6' width=800px>

The basic code to create an agent in LangChain involves:
- defining tools,
- loading a template for ReAct prompting,
- initializing a language model, and
- creating an agent that uses ReAct prompting and language model.
- The agent is then executed using an **`AgentExecutor`**, which *manages the interaction between the agent and the tools*.

This setup allows for dynamic and context-aware responses, enabling the AI to reason and take actions in a structured manner.


### **ReAct Prompting**

Here, we will use an agent that will implement the [ReAct](https://react-lm.github.io/) logic.

**ReAct** overcomes issues of hallucination and error propagation prevalent in chain-of-thought reasoning by interacting with a simple Wikipedia API or any other APIs, and generates human-like task-solving trajectories that are more interpretable than baselines without reasoning traces.

<center><img src='https://react-lm.github.io/files/diagram.png' width=1000px></center>



A **ReAct prompt** consists of few-shot task-solving trajectories, with human-written text reasoning traces and actions, as well as environment observations in response to actions.

ReAct prompting is intuitive and flexible to design, and achieves state-of-the-art few-shot performances across a variety of tasks, from question answering to online shopping.
<br><br>

<img src='https://react-lm.github.io/files/hotpotqa.png' width=1200px>

**Sample ReAct prompt for Agents**

```
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
```

The variables that must be passed as arguments to this prompt are as follows.

- **tools:** A description of the tool, including a description of the tool, input variables for each tool, and their description.
- **input:** question input by agent
- **tools_name:** Names of tools that the agent can use
- **agent_scratchpad:** The agent does not call a tool just once to get the desired answer, but has a structure that calls tools repeatedly until the desired answer is obtained. Each time you call a tool, what the previous call was like, information about the previous call, and the result are stored in this field. It is used to keep track of the agent's thoughts or actions during the execution of the program and let the agent continue its thought process.

The variable values in the prompt are filled in automatically, so there is no place to modify them. As for the prompt, you can also use the example that has already been written. If you just want to improve the agent’s performance, you can modify this prompt.


The prompt guides the agent’s operation mechanism.

```
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
```

At the ***Thought*** stage, the agent uses llm to think about what action to take to get the answer to the question, and ***Action*** decides which tool to use, and ***Action Input*** creates a new question queried by the tool. And finally, ***Observation*** saves the results in the tool called by Action.

Through this sequence, the Agent executes [Thought --> Action --> Action Input --> Observation], saves the result in `agent_scratchpad`, and repeats this process until the desired answer is obtained.

Once you have created a prompt, create an agent using this prompt, and create and call an agent executor using this agent.

If an agent makes a decision about an incoming question and executes a tool, a subject is needed to repeatedly run the agent through this process until an answer is obtained. The agent executor plays this role.

### **Agent execution detailed structure**

The diagram below shows this execution structure graphically.

<img src='https://drive.google.com/uc?id=1c7PXH8bFLIsUawtNes99Ma6BQT24SYXR' width=700px>

When a <font color='#117799'>Question</font> comes in, insert this question into the <font color='#339955'>Agent Prompt</font> created earlier and deliver it to the agent.

To answer a question, the agent refers to the guide in the agent prompt and creates the necessary question (corresponds to <font color='#993333'>Question 1</font>: ***Action Input***) through the thought process. After receiving the answer (corresponding to <font color='#993333'>Answer 1</font>: ***Observation***) by calling the <font color='#339955'>Tool</font> (corresponding to ***Action***) that selected the question, append the answer to `agent_scratchpad` of the existing agent prompt.

The agent executor passes this agent prompt back to the agent, creates <font color='#993333'>Question 2</font> in the same order, receives the answer <font color='#993333'>Answer2</font>, and similarly appends it to the existing agent prompt.

When `agent_executor` passes this prompt to the agent again, if enough information necessary for the answer has been collected, the agent generates an answer based on the information in the prompt and returns the <font color='#117799'>Final Answer</font>.

### Install required dependencies

In [ ]:
%%capture
!pip -q install openai
!pip -q install langchain-core
!pip -q install langchain-community
!pip -q install sentence-transformers
!pip -q install langchain-huggingface
!pip -q install langchain-experimental
!pip -q install langchainhub
!pip -q install langchain-openai

# for wikipedia as a tool
!pip -q install -U wikipedia

# for google-search as a tool
!pip -q install google-search-results

### Import required packages

In [ ]:
import os
from google.colab import userdata

import openai
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import load_tools, create_react_agent, AgentExecutor
from langchain.tools.render import render_text_description
from langchain_community.utilities import SerpAPIWrapper

### **Provide your OpenAI API key**

In [ ]:
# Add your OpenAI key to Colab Secrets with name 'OPENAI_KEY' then run this cell

api_key = userdata.get('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key= os.getenv('OPENAI_API_KEY')
del api_key

### **Load your Language Model**

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# We are going to use this language model as the reasoning engine of the agent to connect to other data sources and perform computation
# so we want this reasoning engine to be as presice as possible.
# Hence, temperature is set to zero to get rid of any randomness.

In [ ]:
response = llm.invoke("How to learn programming? Give 5 points.")
print(response.content)

Learning programming can be an exciting and rewarding journey. Here are five key points to help you get started:

1. **Choose a Programming Language**: Start with a beginner-friendly language such as Python, JavaScript, or Ruby. These languages have extensive resources, community support, and are widely used in various applications.

2. **Utilize Online Resources**: Take advantage of online platforms like Codecademy, freeCodeCamp, Coursera, or edX. These platforms offer structured courses, tutorials, and exercises that can help you learn at your own pace.

3. **Practice Regularly**: Consistent practice is crucial for mastering programming. Work on small projects, solve coding challenges on platforms like LeetCode or HackerRank, and contribute to open-source projects to apply what you’ve learned.

4. **Join a Community**: Engage with other learners and experienced programmers through forums like Stack Overflow, Reddit, or local coding meetups. Participating in a community can provide su

# **LangChain: Agents**

## Built-in LangChain [tools](https://python.langchain.com/v0.2/docs/integrations/tools/)

Here, we will learn how to use

1. Wikipedia tool
2. LLM Math tool
3. Combine two or more tools
4. Google Search Engine tool
5. Python Agent
6. Define your own tool

### **1. Wikipedia tool**

`wikipedia` tool is an api that connects with wikipedia allowing to run search queries against wikipedia and get back the results.

**Load tools**

`load_tools`: Load tools based on their name.

Tools allow agents to interact with various resources and services like APIs, databases, file systems, etc.

To know more about `load_tools` API, refer [here](https://api.python.langchain.com/en/latest/agent_toolkits/langchain_community.agent_toolkits.load_tools.load_tools.html).

In [ ]:
from langchain.agents import load_tools

# Initialize tools
tools = load_tools(["wikipedia"])

tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.10/dist-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000))]

In [ ]:
from langchain.tools.render import render_text_description

# Render tools description
render_text_description(tools)

'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

**Pull ReAct Prompt**

In [ ]:
# Pull the ReAct style prompt template from the hub ; ReAct = Reason and Action
# https://smith.langchain.com/hub/hwchase17/react

from langchain import hub

prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


#### **Create [ReAct](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/) Agent**

`create_react_agent()` creates a ReAct agent using the specified model (llm), tools (tools), and prompt template (prompt).

In [ ]:
# Create the ReAct agent using the create_react_agent function

from langchain.agents import create_react_agent

agent = create_react_agent(
    llm=llm,                         # LLM to use as the agent
    tools=tools,                     # Tools this agent has access to
    prompt=prompt,                   # The prompt to use
    stop_sequence=True,              # stop_sequence=True ensures that the agent stops processing when it reaches a predefined
                                     # stopping point in the conversation. If True, adds a stop token of “Observation:” to avoid hallucinates.
)

In [ ]:
# Create an agent executor from the agent and tools

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,         # Agent to use
    tools=tools,         # Tools to use
    verbose=True,
)

In [ ]:
# Run the agent with a test query

response = agent_executor.invoke(
    {
        "input": "Which of them was built first - Eiffel Tower or Taj Mahal?"
    }
)
response



> Entering new AgentExecutor chain...
To answer the question, I need to find out the construction dates of both the Eiffel Tower and the Taj Mahal. I will look up each structure to determine which one was built first. 

Action: wikipedia  
Action Input: "Eiffel Tower"  Page: Eiffel Tower
Summary: The Eiffel Tower (  EYE-fəl; French: Tour Eiffel [tuʁ ɛfɛl] ) is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower from 1887 to 1889.
Locally nicknamed "La dame de fer" (French for "Iron Lady"), it was constructed as the centerpiece of the 1889 World's Fair, and to crown the centennial anniversary of the French Revolution. Although initially criticised by some of France's leading artists and intellectuals for its design, it has since become a global cultural icon of France and one of the most recognisable structures in the world.  The tower received 5,889,000 visitors in 2022. The Eiff

In [ ]:
# Run the agent with a test query

response = agent_executor.invoke(
    {
        "input": "Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). What book did he write?"
    }
)
response



> Entering new AgentExecutor chain...
I need to find out what book Tom M. Mitchell wrote. 
Action: wikipedia
Action Input: "Tom M. Mitchell book"Page: Tom Clancy
Summary: Thomas Leo Clancy Jr. (April 12, 1947 – October 1, 2013) was an American novelist. He is best known for his technically detailed espionage and military-science storylines set during and after the Cold War. Seventeen of his novels have been bestsellers and more than 100 million copies of his books have been sold. His name was also used on screenplays written by ghostwriters, nonfiction books on military subjects occasionally with co-authors, and video games. He was a part-owner of his hometown Major League Baseball team, the Baltimore Orioles, and vice-chairman of their community activities and public affairs committees.
Originally an insurance agent, Clancy launched his literary career in 1984 when he sold his first military thriller novel The Hunt for Red October for $5,000 published by the small academic Naval Ins

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Ensemble learning
Summary: In statistics and machine learning, ensemble methods use multiple learning algorithms to obtain better predi

{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). What book did he write?',
 'output': 'Tom M. Mitchell wrote the textbook "Machine Learning."'}

### **2. LLM Math tool**

`llm-math` tool is a chain itself which uses a language model in conjuction with a calculater to do math problems.

In [ ]:
tools = load_tools(["llm-math"], llm=llm)

tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question

In [ ]:
render_text_description(tools)

'Calculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.'

In [ ]:
# Create the ReAct agent using the create_react_agent function

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)

In [ ]:
# Create an agent executor from the agent and tools

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [ ]:
# Run the agent with a test query

response = agent_executor.invoke(
    {
        "input": "What is 10% of 500?"
    }
)
response



> Entering new AgentExecutor chain...
To find 10% of 500, I need to calculate 10/100 multiplied by 500. 
Action: Calculator
Action Input: 10/100 * 500Answer: 50.0I now know the final answer.  
Final Answer: 50.0

> Finished chain.


In [ ]:
# Run the agent with a test query

response = agent_executor.invoke(
    {
        "input": "What is 20% of 500?"
    }
)
response



> Entering new AgentExecutor chain...
To find 20% of 500, I need to calculate 20/100 multiplied by 500. 
Action: Calculator
Action Input: 20/100 * 500Answer: 100.0I now know the final answer.  
Final Answer: 100.0

> Finished chain.


### **3. Combining two or more tools**

In [ ]:
tools = load_tools(["llm-math", "wikipedia"], llm=llm)
print(render_text_description(tools))

Calculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.
wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.


In [ ]:
# Create the ReAct agent using the create_react_agent function

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)

In [ ]:
# Create an agent executor from the agent and tools

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [ ]:
question1 = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"

question2 = "What is 20% of 500?"

In [ ]:
result = agent_executor.invoke({"input": question1})
result



> Entering new AgentExecutor chain...
I need to find out what book Tom M. Mitchell has written. Since he is a notable figure in computer science, there may be information available on Wikipedia about his works. 

Action: wikipedia  
Action Input: "Tom M. Mitchell book"  Page: Tom Stoltman
Summary: Tom Ryan Stoltman (born 30 May 1994) is a British professional strongman competitor, from Invergordon, Scotland. He is a three time winner of the World's Strongest Man in 2021, 2022, and 2024. He also won the national title of Britain's Strongest Man on the same years. Nicknamed "The Albatross" due to his sizeable arm span, Stoltman is known for his prowess with the Atlas Stones. In 2020, Stoltman broke the world record for the 5 Atlas Stones (light set) 100–180 kilograms (220–397 lb) in just 16.01 seconds; he also holds the world record for the heaviest Atlas stone ever lifted over a 1.22 metres (4.0 ft) bar at 286 kilograms (631 lb).
On 20 June 2021, Stoltman won the 2021 World's Stronges

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Ensemble learning
Summary: In statistics and machine learning, ensemble methods use multiple learning algorithms to obtain better predi

{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': 'Tom M. Mitchell wrote the book "Machine Learning."'}

In [ ]:
result = agent_executor.invoke({"input": question2})
result



> Entering new AgentExecutor chain...
To find 20% of 500, I need to calculate 20% as a decimal (which is 0.20) and then multiply it by 500. 

Action: Calculator
Action Input: 0.20 * 500Answer: 100.0I now know the final answer.  
Final Answer: 100.0

> Finished chain.


{'input': 'What is 20% of 500?', 'output': '100.0'}

### **4. Google Search Engine**

Certain tools require API Keys. In this example, we use SERPER API.

Create a new account and access your API KEY HERE:
https://serpapi.com/users/welcome

**Provide your SerpAPI key**

In [ ]:
# Add your SerpAPI key to Colab Secrets with name 'SERPAPI_KEY' then run this cell

serp_api_key = userdata.get('SERPAPI_KEY')
os.environ["SERPAPI_API_KEY"] = serp_api_key

del serp_api_key

In [ ]:
# Setup tools

tools = load_tools(["serpapi", "llm-math"], llm=llm)
print(render_text_description(tools))

Search(query: str, **kwargs: Any) -> str - A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.


In [ ]:
# Create the ReAct agent using the create_react_agent function

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)

In [ ]:
# Create an agent executor from the agent and tools

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [ ]:
result = agent_executor.invoke(
    {
        "input": "Which of them was built first - Eiffel Tower or Taj Mahal?"
    }
)
result



> Entering new AgentExecutor chain...
To determine which structure was built first, I need to look up the construction dates of both the Eiffel Tower and the Taj Mahal. 

Action: Search  
Action Input: "Eiffel Tower construction date"  
The assembly of the Tower was a marvel of precision, as all chroniclers of the period agree. The construction work began in January 1887 and was finished on March 31, 1889.I have found the construction date for the Eiffel Tower, which began in January 1887 and was completed in March 1889. Now, I need to find the construction date for the Taj Mahal.

Action: Search  
Action Input: "Taj Mahal construction date"  
1632I have found that the Taj Mahal's construction began in 1632 and was completed in 1648. Now I can compare the two dates.

Final Answer: The Taj Mahal was built first, starting in 1632, while the Eiffel Tower was constructed later, beginning in 1887.

> Finished chain.


In [ ]:
result = agent_executor.invoke(
    {
        "input": "What is the height of Eiffel Tower in meters? Divide the resultant by 2?"
    }
)
result



> Entering new AgentExecutor chain...
To find the height of the Eiffel Tower, I need to recall or search for its height in meters. After that, I will divide that height by 2 to get the final answer. 

Action: Search  
Action Input: "height of Eiffel Tower in meters"  {'type': 'organic_result', 'title': 'Eiffel Tower › Height ShareSend feedbackClose menu'}The height of the Eiffel Tower is approximately 330 meters, including antennas. Now, I will divide this height by 2.

Action: Calculator  
Action Input: 330 / 2  Answer: 165.0I now know the final answer.  
Final Answer: 165.0 meters

> Finished chain.


### **5. Python Agent**

Sometimes, for complex calculations, rather than have an LLM generate the answer directly, it can be better to have the LLM generate code to calculate the answer, and then run that code to get the answer.

The **Python REPL** is a powerful tool for interactive programming, making it easy to experiment with Python code and get immediate feedback.

It allows you to enter Python code, evaluate it immediately, and see the results right away.

Components of Python REPL:

* **Read:** The REPL reads the input you provide. This can be a single line of code, a block of code, or even multiple commands in succession.

* **Eval:** The input is evaluated (executed) by the Python interpreter. This means that the code you entered is processed, and any calculations or operations are performed.

* **Print:** The results of the evaluation are printed to the screen. This allows you to see the output immediately after entering your code.

* **Loop:** After printing the result, the REPL loops back to read the next input, allowing you to continue interacting with the interpreter.

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.utilities import PythonREPL

In [ ]:
python_repl = PythonREPL()

In [ ]:
python_repl.run("print(1+1)")

'2\n'

In [ ]:
# Create the tool to pass to an agent

from langchain_core.tools import Tool

repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

In [ ]:
# Setup tools

tools = load_tools([], llm=llm) + [repl_tool, ]

In [ ]:
# Create the ReAct agent using the create_react_agent function

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)

In [ ]:
# Create an agent executor from the agent and tools

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [ ]:
# Example 1

result = agent_executor.invoke(
    {
        "input": "Sort the following list ['Daisy', 'Elsa', 'Alan', 'Ben', 'Claire'] and print the result"
    }
)
result



> Entering new AgentExecutor chain...
I need to sort the given list of names in alphabetical order. I will use Python's built-in sorting functionality to achieve this. 

Action: python_repl  
Action Input: `sorted(['Daisy', 'Elsa', 'Alan', 'Ben', 'Claire'])`  I need to check the output of the sorting operation to see the sorted list of names. 

Action: python_repl  
Action Input: `print(sorted(['Daisy', 'Elsa', 'Alan', 'Ben', 'Claire']))`  ['Alan', 'Ben', 'Claire', 'Daisy', 'Elsa']
I now know the final answer.  
Final Answer: ['Alan', 'Ben', 'Claire', 'Daisy', 'Elsa']

> Finished chain.


{'input': "Sort the following list ['Daisy', 'Elsa', 'Alan', 'Ben', 'Claire'] and print the result",
 'output': "['Alan', 'Ben', 'Claire', 'Daisy', 'Elsa']"}

In [ ]:
# Example 2

result = agent_executor.invoke(
    {
        "input": "print(len('hello') + 5)"
    }
)
result



> Entering new AgentExecutor chain...
I need to calculate the length of the string 'hello' and then add 5 to that length. The length of 'hello' is 5, so I will perform the addition.

Action: python_repl
Action Input: print(len('hello') + 5)10
I now know the final answer
Final Answer: 10

> Finished chain.


{'input': "print(len('hello') + 5)", 'output': '10'}

### **6. Define your own tool**

First we have to import the tool decorator which convert any function as tool that langchain can use.

In [ ]:
import datetime as dt

def timefunc(text: str) -> str:
    dt_india = dt.datetime.utcnow() + dt.timedelta(hours=5, minutes=30)
    return dt_india.strftime('%Y-%m-%d %H:%M:%S.%f')

In [ ]:
# Create the tool to pass to an agent

from langchain_core.tools import Tool

timetool = Tool(
    name="time_tool",
    description="""Returns current date and time, use this for any \
    questions related to knowing current date and Indian time. \
    The input should always be an empty string, \
    and this function will always return current  \
    date and time.""",
    func = timefunc,
)

In [ ]:
tools = load_tools([], llm=llm) + [timetool, ]
tools

[Tool(name='time_tool', description='Returns current date and time, use this for any     questions related to knowing current date and Indian time.     The input should always be an empty string,     and this function will always return current      date and time.', func=<function timefunc at 0x791fde1ea680>)]

In [ ]:
# Create the ReAct agent using the create_react_agent function

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)

In [ ]:
# Create an agent executor from the agent and tools

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [ ]:
result = agent_executor.invoke(
    {
        "input": "What is the current time?"
    }
)
result



> Entering new AgentExecutor chain...
I need to find out the current date and time. 
Action: time_tool
Action Input: ""2024-10-30 13:30:11.609824I now know the final answer
Final Answer: The current time is 13:30:11 on October 30, 2024.

> Finished chain.


{'input': 'What is the current time?',
 'output': 'The current time is 13:30:11 on October 30, 2024.'}